In [ ]:
!pip install requests beautifulsoup4 fake-useragent
import asyncio
import requests
from bs4 import BeautifulSoup
import base64
from fake_useragent import UserAgent
import os

biskit_link = "https://biskit.kumoh.ac.kr"
session = requests.Session()
headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
'User-Agent': UserAgent().random,
}
session.headers.update(headers)

def get_jsessionid():
    jsessionid4_url = "https://biskit.kumoh.ac.kr"
    jsessionid4_res = session.get(jsessionid4_url)

    JSESSIONID4 = jsessionid4_res.cookies['JSESSIONID4']
    WMONID = jsessionid4_res.cookies['WMONID']

    logout_url = "https://biskit.kumoh.ac.kr/common/user/logout.do"
    session.headers.update({'Cookie': f'JSESSIONID4={JSESSIONID4}; WMONID={WMONID};'})
    logout_res = session.get(logout_url, allow_redirects=False)

    login_url = "https://biskit.kumoh.ac.kr/common/user/login.do"

    session.headers.update({'Cookie': f'JSESSIONID4={JSESSIONID4}; WMONID={WMONID};'})
    login_res = session.get(login_url)
    JSESSIONID4 = login_res.cookies['JSESSIONID4']

    jsessionid_url = "https://onekit.kumoh.ac.kr/index.jsp"
    session.headers.update({'Cookie': f'JSESSIONID4={JSESSIONID4};'})
    jsessionid_res = session.get(jsessionid_url, allow_redirects=False)
    JSESSIONID = jsessionid_res.cookies['JSESSIONID']

    return JSESSIONID4, JSESSIONID, WMONID

def biskit_login():
    JSESSIONID4, JSESSIONID, WMONID = get_jsessionid()

    Cookie = f'WMONID={WMONID}; JSESSIONID4={JSESSIONID4}; JSESSIONID={JSESSIONID};'
    biskit_Cookie = f'WMONID={WMONID}; JSESSIONID4={JSESSIONID4}; G_ENABLED_IDPS=google;'

    first_url = "https://biskit.kumoh.ac.kr/sso/index.jsp"

    session.headers.update({'Cookie': biskit_Cookie})

    first_res = session.get(first_url, allow_redirects=False)

    second_url = first_res.headers['Location']
    session.headers.update({'Cookie': Cookie})
    second_res = session.get(second_url, allow_redirects=False)

    ORIGINAL_JESSIONID = second_res.cookies['JSESSIONID']

    login_url = "https://onekit.kumoh.ac.kr/proc/Login.eps"

    login_data = {
        'user_id': '2025114794',
        'user_password':  'qwe123!!',
    }
    session.headers.update({'Cookie': Cookie})
    login_res = session.post(login_url, data=login_data, allow_redirects=False)

    JSESSIONID = login_res.cookies['JSESSIONID']
    Cookie = f'WMONID={WMONID}; JSESSIONID4={JSESSIONID4}; JSESSIONID={JSESSIONID};'

    sso_url = login_res.headers['Location']
    session.headers.update({'Cookie': f"JSESSIONID={ORIGINAL_JESSIONID}; JSESSIONID4={JSESSIONID4};"})
    sso_res = session.get(sso_url, allow_redirects=False)

    biskit_url = "https://biskit.kumoh.ac.kr/sso/index.jsp"

    session.headers.update({'Cookie': biskit_Cookie})
    biskit_res = requests.get(biskit_url, allow_redirects=False)

    biskit_sso_url = biskit_res.headers['Location']
    session.headers.update({'Cookie': f"JSESSIONID4={JSESSIONID4}; JSESSIONID={ORIGINAL_JESSIONID}; _SSO_Global_Logout_url=get%5Ehttps%3A%2F%2Fonekit.kumoh.ac.kr%2Flogout.jsp%3Flogout%3D1%24"})
    biskit_sso_res = session.get(biskit_sso_url, allow_redirects=False)

    biskit_sso_1_url = biskit_sso_res.headers['Location']

    session.headers.update({'Cookie': biskit_Cookie})
    biskit_sso_1_res = session.get(biskit_sso_1_url, allow_redirects=False)

    biskit_login_url = biskit_sso_1_res.headers['Location']

    session.headers.update({'Cookie': biskit_Cookie})
    biskit_login_res = session.get(biskit_login_url, allow_redirects=False)

    biskit_sso_2_url = biskit_login_res.headers['Location']

    session.headers.update({'Cookie': biskit_Cookie})
    biskit_sso_2_res = session.get(biskit_sso_2_url, allow_redirects=False)

    biskit_sso_2_soup = BeautifulSoup(biskit_sso_2_res.text, 'html.parser')
    userId = biskit_sso_2_soup.find('input', {'name': 'userId'})['value']

    biskit_login_proc_url = "https://biskit.kumoh.ac.kr/common/user/loginProc.do"

    biskit_login_proc_data = {
        'userId': userId,
        'rtnUrl': ''
    }
    session.headers.update({'Cookie': biskit_Cookie})
    biskit_login_proc_res = session.post(biskit_login_proc_url, data=biskit_login_proc_data, allow_redirects=False)

    return biskit_Cookie

In [ ]:

def get_preview(post_id: int) -> tuple:
    biskit_Cookie = biskit_login()
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
            'Cookie': biskit_Cookie}

    link = f"{biskit_link}/ptfol/imng/icmpNsbjtPgm/findIcmpNsbjtPgmInfo.do?encSddpbSeq={post_id}"

    html = requests.get(link, headers=header).text
    soup = BeautifulSoup(html, 'html.parser')

    text_list = soup.find('div', {"class": "table_wrap"}).find('tbody')
    text_list = soup.find('div', {"class": "table_wrap"}).find('tbody').find_all('tr')[-1].find_all('p')

    # Set img preview
    img_preview = None
    img_preview_base64 = None
    try:
        img_preview = biskit_link + soup.find('div', {"class": "table_wrap"}).find('tbody').find('img')['src']
        res = requests.get(img_preview, headers=header)
        with open('./biskit_temp.png', 'wb') as f:
            f.write(res.content)

        with open('./biskit_temp.png', 'rb') as f:
            base64_str = base64.b64encode(f.read())

        img_preview_base64 = base64_str

        os.remove('./biskit_temp.png')
    except:
        pass

    text = ''
    for i in text_list:
        text += i.getText().replace(r'\xa0', '') + " "

    if len(text) <= 100:
        result = text
    else:
        result = f'{text[:100]} ...[더보기]({biskit_link}ptfol/imng/icmpNsbjtPgm/findIcmpNsbjtPgmInfo.do?encSddpbSeq={post_id})'

    title = soup.find('div', {"class": "tab_top_wrap"}).find('h4').text.strip()
    for i in soup.find('div', {"class": "table_wrap"}).find('tbody').find_all('tr'):
        
        if not i.find('th'):
            continue

        if i.find('th').text.strip() == "운영조직":
            org = i.find_all('td')[0].text.strip()
            author = i.find_all('td')[1].text.strip()

        elif i.find('th').text.strip() == "프로그램 분류":
            category = i.find('td').text.strip()

        elif i.find('th').text.strip() == "신청기간":
            period = i.find('td').text.strip().replace(r'\xa0', ' ')

        elif i.find('th').text.strip() == "수료 인증서":
            mileage = int(i.find_all('td')[1].text.strip())
            
    post = (0, post_id, title, author, org, category, period, mileage)
    return post, img_preview_base64, result
# test
if __name__ == "__main__":
    print(get_preview(31303033323734))



((0, 31303033323734, '[공학] ROS기반 자율주행 교육과정', '김혜진(0544787952)', '공학교육혁신센터', '', '2025.07.15 10:00\xa0~\xa02025.07.23 18:00', 264), None, '15:00~17:00 경주 → 이동 (구미) ')
